In [23]:
%pip install nltk

In [24]:
import pandas as pd
from nltk import trigrams
import numpy as np
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import KneserNeyInterpolated
from nltk import everygrams
from nltk.lm.preprocessing import pad_both_ends
import math


In [25]:
df = pd.read_csv('gigaword.csv', nrows=1000)
# give the column names
df.columns = ['summary']


tokenized_text = [sentence.lower().split() for sentence in df['summary']]


train_data, padded_vocab = padded_everygram_pipeline(3, tokenized_text)

model = KneserNeyInterpolated(order=3)
model.fit(train_data, padded_vocab)

In [26]:
def calculate_A1_fluency(sentence, model):
    if not sentence:
        return 0

    split_sentence = sentence.lower().split()
    # Pad the sentence
    padded_sentence = list(pad_both_ends(split_sentence, n=3))
    # Get all the trigrams
    trigrams_in_sentence = list(trigrams(padded_sentence))
    # Calculate the probability of the sentence
    sentence_log_probability = 0.0
    for trigram in trigrams_in_sentence:
        trigram_probability = model.score(trigram[2], (trigram[0], trigram[1]))
        sentence_log_probability += math.log(max(trigram_probability, 1e-10))
    return sentence_log_probability/len(trigrams_in_sentence)
     # Calculate the product of probabilities for each trigram
    # for trigram in sentence_trigrams:
    #     *context, word = trigram
    #     trigram_prob = model.score(word, context)
    #     # Safeguard against log(0) by using a minimum probability threshold
    #     probability *= max(trigram_prob, 1e-10)

    # return probability


In [27]:
def getValuesOfGoodAndBad(model):
    # get all the conditional probabilities of the model
    conditional_probabilities = model.context_counts
    # sort them in descending order
    sorted_conditional_probabilities = sorted(conditional_probabilities.items(), key=lambda x: x[1], reverse=True)
    # lowest CP of the first forty percent is assigned to  value for good
    # highest CP of the last twenty percent is assigned to value for bad
    good = sorted_conditional_probabilities[int(len(sorted_conditional_probabilities)*0.4)][1]
    bad = sorted_conditional_probabilities[int(len(sorted_conditional_probabilities)*0.8)][1]
    return good, bad


In [28]:
# valForGood ,valForBad = getValuesOfGoodAndBad(model)
valForGood = 0.0000001
valForBad =  0.000000001
def calculate_A2_fluency(sentence, model):
    if not sentence:
        return 0
    split_sentence = sentence.lower().split()
    # Pad the sentence
    padded_sentence = list(pad_both_ends(split_sentence, n=3))
    # Get all the trigrams
    trigrams_in_sentence = list(trigrams(padded_sentence))
    # Calculate the probability of the sentence
    fluency = 1.0/len(trigrams_in_sentence)
    for trigram in trigrams_in_sentence:
        trigram_probability = model.score(trigram[2], (trigram[0], trigram[1]))
        if trigram_probability >= valForGood:
            fluency = fluency/trigram_probability
        elif trigram_probability <= valForBad:
            fluency = fluency*trigram_probability
    return fluency

In [29]:
df['A1_fluency_score'] = df['summary'].apply(lambda x: calculate_A1_fluency(x, model))
df['A2_fluency_score'] = df['summary'].apply(lambda x: calculate_A2_fluency(x, model))

df.to_csv('A1A2_fluency_scores.csv', index=False)